# Energy renovation project

#### Package import

In [6]:
import random
import numpy as np
import pandas as pd

#### Inputs
Here you can change the values for investment costs, number of building elements and the m number, which is defined for the normalized problem as the maximum count of measures over all building elements.

In [7]:
# The total renovation investment costs
Inv = 1000
# the total number of building elements
n = 10
# the maximum count of measures over all building elements for a normalized problem 
m = 5

#### Parameters of the measures 
Values related to material costs and heat loss costs after 20 years of usage are set randomly for experimental purposes. Their values can also be loaded from other data sources, such as CSV-files.

##### Experimental runs
- Material costs are taken with 1 decimal point and generated using samples from a uniform distribution from low value of 50 to the highest value of 300 price unit
- Heat loss costs after 20 years of usage from 10 to 1000 units

In [8]:
d = {"e": [], "Kij": [], "Hij": []}
for i in range(n):
    d["e"].append(i)
    # Material costs
    d["Kij"].append(np.round(np.random.uniform(low=50, high=300, size=(m-1,)), decimals=1))
    # Heat loss costs TODO("Rethink low and high values")
    d["Hij"].append(np.round(np.random.uniform(low=10, high=1000, size=(m,)), decimals=1))